In [10]:
from pyspark.sql import SparkSession
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.classification import DecisionTreeClassificationModel
from pyspark.ml.linalg import Vectors
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import regexp_replace
#DecisionTreeClassifier


In [2]:
feature_pipe = PipelineModel.load('feature_pipe')
model = DecisionTreeClassificationModel.load('model_dct')
spark = SparkSession.builder.master('local').appName('deployPyspark').getOrCreate


Picked up _JAVA_OPTIONS: -Xmx12288m
Picked up _JAVA_OPTIONS: -Xmx12288m
23/05/23 04:07:33 WARN Utils: Your hostname, joanPC resolves to a loopback address: 127.0.1.1; using 192.168.1.201 instead (on interface wlo1)
23/05/23 04:07:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/23 04:07:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
columns=['Tipo de Proceso','Objeto a Contratar','Tipo de Contrato','Nombre de la Entidad','Nombre Grupo','Nombre Clase','Plazo de Ejec del Contrato','Rango de Ejec del Contrato','Cuantia Contrato','Origen de los Recursos','Municipio Entidad','Departamento Entidad']

In [4]:
columns

['Tipo de Proceso',
 'Objeto a Contratar',
 'Tipo de Contrato',
 'Nombre de la Entidad',
 'Nombre Grupo',
 'Nombre Clase',
 'Plazo de Ejec del Contrato',
 'Rango de Ejec del Contrato',
 'Cuantia Contrato',
 'Origen de los Recursos',
 'Municipio Entidad',
 'Departamento Entidad']

### Diccionario para seleccionar las caracteristicas del contrato

In [ ]:
tp=data_2.select('Tipo de Proceso').distinct().rdd.map(lambda x : x[0]).collect()
oc=data_2.select('Objeto a Contratar').distinct().rdd.map(lambda x : x[0]).collect()
tc=data_2.select('Tipo de Contrato').distinct().rdd.map(lambda x : x[0]).collect()

ng=data_2.select('Nombre Grupo').distinct().rdd.map(lambda x : x[0]).collect()
nc=data_2.select('Nombre Clase').distinct().rdd.map(lambda x : x[0]).collect()
#plazo eject
re=data_2.select('Rango de Ejec del Contrato').distinct().rdd.map(lambda x : x[0]).collect()
#cuantia con
orec=data_2.select('Origen de los Recursos').distinct().rdd.map(lambda x : x[0]).collect()
de=data_2.select('Departamento Entidad').distinct().rdd.map(lambda x : x[0]).collect()

In [90]:
dic={'tipo_proceso':tp,'objeto_contratar':oc,'tipo_contrato':tc,'nombre_grupo':ng,'nombre_clase':nc,'rango_ejec':re,
    'origen_recursos':orec,'departamento':de}

In [96]:
import numpy as np
np.save('dic.npy', dic)


In [105]:
new_dict = np.load('dic.npy',allow_pickle='TRUE').item()

In [119]:
s=new_dict['origen_recursos'][2]


In [125]:
type(s)

str

### Archivo en excel para filtrar nombre de la entidad por Municipio y el Municipio por Departamento.

In [70]:
mundep=data_2.select('Nombre de la Entidad','Municipio Entidad','Departamento Entidad').distinct()

In [78]:

mundep.repartition(1).write.csv("entidades.csv",
              header='True', sep='|')


In [81]:
import pandas as pd
entidades=pd.read_csv("entidades2.csv",sep='|')

In [87]:
en=entidades[entidades['Departamento Entidad']=='Antioquia']
l=en['Municipio Entidad'].unique()

In [ ]:
mune=entidades[entidades['Municipio Entidad']=='Medellin']
lent=en['Nombre de la Entidad'].unique()

In [142]:

from pyspark.sql.functions import col
data_2.filter(col("Nombre de la Entidad") == 'ANTIOQUIA  ALCALDÍA MUNICIPIO DE HELICONIA').show()

#ANTIOQUIA ALCALDÍA MUNICIPIO DE MEDELLÍN

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------------+--------------------------+----------------+----------------------+-----------------+--------------------+
|     Tipo de Proceso|  Objeto a Contratar|    Tipo de Contrato|Nombre de la Entidad|        Nombre Grupo|        Nombre Clase|Plazo de Ejec del Contrato|Rango de Ejec del Contrato|Cuantia Contrato|Origen de los Recursos|Municipio Entidad|Departamento Entidad|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------------+--------------------------+----------------+----------------------+-----------------+--------------------+
|Contratación Dire...|Organizaciones y ...|Prestación de Ser...|ANTIOQUIA  ALCALD...|       [F] Servicios|     Clubes sociales|                         4|                         M|         8800000|           No Defin

In [5]:
spark = SparkSession.builder.getOrCreate()
sdf = spark.read.options(inferSchema='True',
              header='True').csv('SECOP_1_.csv')

In [16]:
data_3=sdf.withColumn("Cuantia Contrato",regexp_replace("Cuantia Contrato", ",", ""))


In [17]:
data_3 = data_3.distinct()

In [18]:
data_3=data_3.na.drop(how="any")

In [19]:
data_3=data_3.withColumn("Cuantia Contrato",data_3["Cuantia Contrato"].cast(IntegerType()))

In [20]:
data_3=data_3.withColumn("Valor Total de Adiciones",regexp_replace("Valor Total de Adiciones", ",", ""))

In [21]:
data_3=data_3.withColumn("Valor Total de Adiciones",data_3["Valor Total de Adiciones"].cast(IntegerType()))

In [22]:
from pyspark.sql.functions import col
data_3=data_3.withColumn("Porcentaje_Adiciones",col("Valor Total de Adiciones")/col("Cuantia Contrato")*100)

In [23]:
import pyspark.sql.functions as func
data_3 = data_3.withColumn("Porcentaje_Adiciones", func.round(data_3["Porcentaje_Adiciones"], 2))

In [24]:
data_3=data_3.filter(col("Cuantia Contrato") >= 1000000)

In [25]:
data_3=data_3.filter((col('Valor Total de Adiciones') >= 100000) | (col('Valor Total de Adiciones')==0))

In [55]:
difilt=data_3.filter((col("Porcentaje_Adiciones") > 50) & (col("Cuantia Contrato") >= 2000000000))

In [56]:

difilt.select('Ruta Proceso en SECOP I','Tipo de Proceso','Objeto a Contratar',
      'Tipo de Contrato','Nombre de la Entidad',
       'Nombre Grupo', 'Nombre Clase','Plazo de Ejec del Contrato',
        'Rango de Ejec del Contrato','Cuantia Contrato',
       'Origen de los Recursos',  'Municipio Entidad', 'Departamento Entidad',"Porcentaje_Adiciones").show(truncate=False)

+------------------------------------------------------------------------------------+-------------------------------------------------------+------------------------------------------------------------------------------------+-----------------------+-------------------------------------------------------------------------+-------------------------------------------+----------------------------------------------------------------+--------------------------+--------------------------+----------------+----------------------+-----------------+--------------------+--------------------+
|Ruta Proceso en SECOP I                                                             |Tipo de Proceso                                        |Objeto a Contratar                                                                  |Tipo de Contrato       |Nombre de la Entidad                                                     |Nombre Grupo                               |Nombre Clase                              

23/05/25 08:16:02 WARN TransportChannelHandler: Exception in connection from /192.168.1.201:45249
java.io.IOException: Expiró el tiempo de conexión
	at java.base/sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at java.base/sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at java.base/sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:276)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:233)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:223)
	at java.base/sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:356)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:259)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:788)
	at io.netty.channel.nio.NioEventLoop.processSelect